# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Dataset, Experiment, Environment, ScriptRunConfig
from azureml.core.webservice import Webservice
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, uniform
from azureml.widgets import RunDetails

import logging
import joblib


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

experiment_name = 'hyperdrive-barrios-madrid'

experiment=Experiment(ws, experiment_name)

dataset = Dataset.get_by_name(ws, name='panel_indicadores_distritos_barrios_2022')
df = dataset.to_pandas_dataframe()
df.head(5)

,Barrio,Año medio de contrucción de inmuebles de uso residencial,Apartamentos Municipales para Mayores,Asociaciones (Sección 1ª),Asociaciones culturales y casas regionales,Asociaciones de mujeres,Asociaciones vecinales,Bibliotecas Comunidad Madrid,Bibliotecas Municipales,Campos de fútbol 11,...,Tasa absoluta de paro registrado Mujeres,Tasa bruta de natalidad (‰),Tasa de crecimiento demográfico (porcentaje),Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Total hogares
0,11,1928,1,49,22,2,1,0,1,0,...,7.24,6.4,0.0,3.00,5.01,11.38,1.48,6.28,9.96,11613
1,12,1927,0,98,28,4,9,1,0,0,...,8.29,6.3,0.4,3.71,5.31,10.54,4.23,6.96,11.46,22614
2,13,1923,0,31,10,1,1,0,0,0,...,5.93,5.9,0.2,0.90,3.45,8.73,0.79,4.91,8.78,5488
3,14,1929,0,36,10,4,1,0,1,0,...,5.26,6.8,1.0,1.28,3.76,9.13,0.64,3.56,8.85,8927
4,15,1930,0,46,16,1,2,0,1,0,...,6.32,5.6,0.0,2.63,4.57,9.34,1.20,4.59,10.36,16967


In [4]:
ws

Workspace.create(name='quick-starts-ws-242700', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-242700')

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2,
                                        slack_factor=0.1,
                                        delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    'max_depth': choice(5, 10, 15, 20, 25),
    'min_samples_split': uniform(0.001, 0.5),
    'min_samples_leaf': uniform(0.001, 0.5)
})

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yaml')

#TODO: Create your estimator and hyperdrive config
src = ScriptRunConfig(source_directory='.', script='train.py',
                      arguments=['--max_depth', 15, '--min_samples_split', 0.1, '--min_samples_leaf', 0.1],
                      compute_target=ComputeTarget(workspace=ws, name='hyperdrive-compute'),
                      environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='r2',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4
                    )

In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config) 

In [16]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_cc3ad8e1-0a64-46cb-ac6c-a78a999e1ce9
Web View: https://ml.azure.com/runs/HD_cc3ad8e1-0a64-46cb-ac6c-a78a999e1ce9?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-242700/workspaces/quick-starts-ws-242700&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-10-13T11:48:32.693438][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-10-13T11:48:33.1846143Z][SCHEDULER][INFO]Scheduling job, id='HD_cc3ad8e1-0a64-46cb-ac6c-a78a999e1ce9_0' 
[2023-10-13T11:48:33.2926220Z][SCHEDULER][INFO]Scheduling job, id='HD_cc3ad8e1-0a64-46cb-ac6c-a78a999e1ce9_1' 
[2023-10-13T11:48:33.4097171Z][SCHEDULER][INFO]Scheduling job, id='HD_cc3ad8e1-0a64-46cb-ac6c-a78a999e1ce9_2' 
[2023-10-13T11:48:33.5509681Z][SCHEDULER][INFO]Scheduling job, id='HD_cc3ad8e1-0a64-46cb-ac6c-a78a999e1ce9_3' 
[2023-10-13T11:48:33.464576][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
!jupyter nbextension enable azureml.widgets --user --py

Enabling notebook extension azureml_widgets/extension...
      - Validating: OK


In [11]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run, fitted_model = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run_parameters = best_run.get_details()['runDefinition']['parameters']
print(best_run_parameters)

In [ ]:
#TODO: Save the best model
joblib.dump(fitted_model, 'hyperdrive_model.pkl')

model_name= best_run.properties['model_name']
description = 'Best model trained using Hyperdrive'
tags = None
model = hyperdrive_run(model_name=model_name,
                       description=description,
                       tags=tags)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

